# Predict how long a trip will take
# Predict how much the trip will cost

- predicting duration in seconds
    - why is this useful???
    - SHIT, fees are restrospective features
    - if this were to be in production, which features do we feed our model??
- predicting fare amount a better problem???

- measure error with rmsle
- train with May 2019
- test with June 2019

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data

In [10]:
df = pd.read_csv('datasets/green_tripdata_2019-05.csv', parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime'])
df = df[df.RatecodeID == 1]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493058 entries, 0 to 504886
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               493058 non-null  int64         
 1   lpep_pickup_datetime   493058 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  493058 non-null  datetime64[ns]
 3   store_and_fwd_flag     493058 non-null  object        
 4   RatecodeID             493058 non-null  int64         
 5   PULocationID           493058 non-null  int64         
 6   DOLocationID           493058 non-null  int64         
 7   passenger_count        493058 non-null  int64         
 8   trip_distance          493058 non-null  float64       
 9   fare_amount            493058 non-null  float64       
 10  extra                  493058 non-null  float64       
 11  mta_tax                493058 non-null  float64       
 12  tip_amount             493058 non-null  floa

In [22]:
location_lookup = pd.read_csv('datasets/taxi+_zone_lookup.csv')
location_lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


seems like there are some hidden nulls

# Look at features

- Pickup & drop off time
    - use these two features to derive our target variable = duration
    - can derive hour of the day that can be useful
- Location ID
    - can we tell much about these??
- RateCodeId
    - we'll look at 'Standard rate' only
    - other code will have different fees calculation
- fees related stuff should be correlated in some way, look to select only a few
    - fare_amount - calculated by meter
    - extra - 0.5 for rush hour and 1 for overnight (let's try to binarize these)
    - tip_amount - cash tips not included
    - tolls_amount - tools fee
    - mta_tax, improvement_surcharge --> constant
    - ehail_fee all null
    - congestion_surcharge - can this help indicate traffic

    - total_amount = total that customer paid except tips (can only use this for basic model)


- Discard some features
    - VendorID
        - let's assume that they all have similar drivers and release schedules
        - this may be an issue if some vendors have only night shifts = less traffic
    - store_and_fwd_code
        - purely about data submission
    - RateCodeId
        - we've already used this for filtering (we currently focus on Standard rate)
    - passenger_count
        - should have no influence on trip distance????????
    - mta_tax, improvement_surcharge
        - constant
    - tip_amount
        - remove because cash trips are not included in the data
    - payment type
    - trip_type
        - street hail or dispatch should not have influence on duration
        - duration is calculated from 

In [12]:
BASIC_MODEL = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

# Basic model

In [18]:
df_basic = df[BASIC_MODEL]

## Feature engineering
- derive target variable
- join location id to lookup table
    - and get borough name only??
    - will this help much??
- weekend binary
- rush hour, working hours, overnight bins

## EDA

In [19]:
# sample for EDA
df_basic_sample = df_basic.sample(n=10000, random_state=9)

In [ ]:
# trip distance

In [ ]:
# number of trips

In [20]:
# number of trips by

# date of month (end of month people use cab more, or run out of money use subway haha)
# day of week
# weekdays and weekends

In [23]:
# Pickup and drop off location by boroughs
# Do people go cross boroughs with taxis?? indicating longer distances

## Fit basic model